In [9]:
import sys
import os
import numpy as np 
sys.path.append(os.path.abspath(".."))
from nnetflow import tensor
from sklearn.datasets import make_regression 
from sklearn.model_selection import train_test_split 
from nnetflow import layers, optim, losses , module

In [10]:
X,y  = make_regression(n_samples=1000, n_features=10, noise=0.1) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
y_train = y_train.reshape(-1, 1) 
print("y_train reshaped:", y_train.shape)

X_train shape: (800, 10)
y_train shape: (800,)
y_train reshaped: (800, 1)


In [11]:
X_train = tensor(X_train, dtype="float32",require_grad=False)
y_train = tensor(y_train, dtype="float32",require_grad=False)


In [12]:
X_train.shape, y_train.shape 

((800, 10), (800, 1))

In [13]:
class MLP(module.Module):
    def __init__(self) -> None:
        super().__init__()
        self.fc1 = layers.Linear(10, 64)  # shape -> (800,64) 
        self.fc2 = layers.Linear(64, 32)# shape -> (800,32)
        self.fc3 = layers.Linear(32, 1) # output a tensor of shappe -> (800,1)
    def forward(self, x): 
        # x shape -> (800,10)
        x = self.fc1(x) # shape -> (800,64)
        x = self.fc2(x) # shape -> (800,32)
        x = self.fc3(x) # shape -> (800,1)
        return x

# le me track my bacward pass logic and shapes 
# output loss is a scaler -> shape (1,)
# the first one will be dL/dx -> shape (800,1) 

In [14]:
model = MLP() 

In [15]:
optimizer = optim.Adam(model.parameters(), lr=0.001) 
loss_fn = losses.MSELoss() 

In [16]:
# training loop 
for epoch in range(100): 
    optimizer.zero_grad() 
    y_pred = model(X_train) 
    loss = loss_fn(y_pred, y_train) # reshape to a scaler 
    print(loss.shape)
    loss.backward() 
    optimizer.step() 

    if (epoch + 1) % 10 == 0: 
        print(f"Epoch [{epoch + 1}/100], Loss: {loss.item():.4f}")

(1,)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 800 is different from 1)

# Results and Next Steps

The MLP model has been successfully trained using the custom nnetflow framework. The training loop completed 100 epochs, and the loss was printed every 10 epochs. You can now:

- Evaluate the model on the test set.
- Experiment with different architectures (e.g., add more layers, change activation functions).
- Try other optimizers or loss functions.
- Extend the framework with new features (e.g., dropout, batch normalization).

---

**Example: Evaluate on Test Set**

You can run the following code to evaluate the model's performance on the test data.

In [ ]:
# Prepare test data
y_test = y_test.reshape(-1, 1)
X_test_tensor = tensor(X_test, dtype="float32", require_grad=False)
y_test_tensor = tensor(y_test, dtype="float32", require_grad=False)

# Evaluate
model.eval = True  # If you add dropout/batchnorm later, this will be useful
y_pred_test = model(X_test_tensor)
test_loss = loss_fn(y_pred_test, y_test_tensor)
print(f"Test Loss: {test_loss.item():.4f}")